In [1]:
import pandas as pd
from sklearn.utils import shuffle

train_all = pd.read_csv("C:/Users/Desktop/UDSC/Sentiment analysis/Data/Amazon/train.csv", sep="|")
test_Amazon = pd.read_csv("C:/Users/Desktop/UDSC/Sentiment analysis/Data/Amazon/test.csv", sep="|")
test_MR = pd.read_csv("C:/Users/Desktop/UDSC/Sentiment analysis/Data/Amazon/movie reviews.csv", sep="|")

train_all = train_all.rename(columns={'overall' : 'label', 'reviewText' : 'text'})
test_Amazon = test_Amazon.rename(columns={'overall' : 'label', 'reviewText' : 'text'})

test_Amazon = test_Amazon.astype(str)
test_MR = test_MR.astype(str)

In [2]:
train_all_pos = train_all[train_all.label == 1]
train = pd.concat([train_all_pos.sample (n=185412, random_state=42), train_all[train_all.label == 0]])
train = shuffle(train,random_state=42)

In [3]:
print(train.shape)
train.groupby('label').count()

(370822, 2)


,text
label,
0,185402
1,185402


In [4]:
train = train.astype(str)

In [5]:
import re, nltk
from nltk.stem import SnowballStemmer
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    word_list = nltk.word_tokenize(text)
    stemmer = SnowballStemmer("english")
    stems = [stemmer.stem(word) for word in word_list]
    return stems

In [8]:
%%time

from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, stop_words = 'english')

data_features = vectorizer.fit_transform(train['text'])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        train.text, 
        train.label,
        test_size=0.1, 
        random_state=42)


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('vectorizer', HashingVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, stop_words = 'english')), 
                     ('classifier', LogisticRegression(C = 100))])

model = pipeline.fit(X=X_train, y=y_train)

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print ("Validation Accuracy Amazon :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

X_test_MR, y_test_MR = (test_MR.text, test_MR.label)
y_pred_MR = model.predict(X_test_MR)

print ("Test Accuracy MR :", accuracy_score(y_test_MR, y_pred_MR))
print(classification_report(y_test_MR, y_pred_MR))

X_test_amazon, y_test_amazon = (test_Amazon.text, test_Amazon.label)
y_pred_amazon = model.predict(X_test_amazon)

print ("Test Accuracy Amazon :", accuracy_score(y_test_amazon, y_pred_amazon))
print(classification_report(y_test_amazon, y_pred_amazon))

Validation Accuracy Amazon : 0.896772105817
             precision    recall  f1-score   support

          0       0.90      0.89      0.90     18600
          1       0.89      0.90      0.90     18483

avg / total       0.90      0.90      0.90     37083

Test Accuracy MR : 0.748592870544
             precision    recall  f1-score   support

          0       0.75      0.74      0.75      5330
          1       0.74      0.76      0.75      5330

avg / total       0.75      0.75      0.75     10660

Test Accuracy Amazon : 0.900190463681
             precision    recall  f1-score   support

          0       0.59      0.89      0.71     21219
          1       0.98      0.90      0.94    131041

avg / total       0.93      0.90      0.91    152260

Wall time: 58min 27s
